# spaCy para análise de texto

O spaCy é Framework de Processamento de linguagem natural (NLP).

NLP = analisar linguagem humana através de um sistema de computador, também chamado de linguística computacional.

Com spaCy podemos analisar as estruturas textuais, classes gramaticais, reconhecer entidades, analisar as relações entre os termos, etc.


O que vamos aprender hoje:

* o que é o spaCy;
* instalar, carregar modelos
* Criar um `Doc Container`
* Trabalhar com sentenças
* Entender o que é um `token` e seus atributos
* Aplicar reconhecimento de entidades.

## Instalando o spaCy

In [ ]:
!pip install spacy

## Carregando modelos

In [1]:
import spacy


In [2]:
!python -m spacy download pt_core_news_sm

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 21.9 MB 4.4 MB/s 
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [3]:
# carregar o modelo que foi previamente instalado, nesse caso 
# o modelo 'pt_core_news_sm' = modelo de português reduzido
nlp = spacy.load('pt_core_news_sm')

### O que são esses modelos?

Ver a [documentação do spaCy](https://spacy.io/models/pt)

## Criar um `Doc Container`

In [4]:
# ler o texto
text = open('wlamyra.txt', 'r').read()

In [5]:
# processar o texto
doc = nlp(text)

## Trabalhar com sentenças

In [6]:
# mostrar sentenças
doc.sents


In [7]:
# ver as sentenças do texto
print([sent.text for sent in doc.sents])


['\nWlamyra Albuquerque\n\n', 'Quem anda com porcos...', 'Ou negacionismo vende anúncio?', '\n30 de setembro de 2021\n\n', 'Quem anda com porcos, farelo come.', 'Este é um velho dito popular ainda útil para quem lê na Folha textos que disparam paisagens negacionistas e ideias racistas e, ainda assim, têm espaço no jornal de maior circulação no país.', 'Não vou gastar um minuto deste dia ensolarado para debater a interpretação rasteira, rasa mesmo, que certo jornalista fez do escravismo e da agência de mulheres escravizadas no Brasil.', '\n\nÉ "texto-farelo".', 'Historiadoras estão acostumadas a sentar-se à mesa dos debates marcados por divergências bem articuladas e fundamentadas.', 'É banquete para nós que temos discordâncias fundadas em pesquisas científicas e em leituras honestas dos textos alheios.', '\n\n', 'O artigo do colunista Leandro Narloch ("Luxo e riqueza das \'sinhás pretas\' precisam inspirar o movimento negro", 29/9) não passa de farelo à mesa para leitores que merecem a

In [8]:
doc.sents[11].text

TypeError: 'generator' object is not subscriptable

In [9]:
# transformar o objeto doc.sents em uma lista de strings
sents = list(doc.sents)

In [12]:
sents[11].text

'O artigo do colunista Leandro Narloch ("Luxo e riqueza das \'sinhás pretas\' precisam inspirar o movimento negro", 29/9) não passa de farelo à mesa para leitores que merecem algo melhor.'

## Tokens

In [13]:
print (len(doc))
print (len(text))

846
4604


In [14]:
# imprimir os 10 primeiros tokens
for token in doc[:10]:
    print(token)



Wlamyra
Albuquerque



Quem
anda
com
porcos
...
Ou


In [15]:
# e se fizermos isso com a variável text?
for i in text[:10]:
    print (i)



W
l
a
m
y
r
a
 
A


### Atributos de um token

* .text
* .head
* .left_edge
* .right_edge
* .ent_type_
* .iob_
* .lemma_
* .morph
* .pos_
* .dep_
* .lang_


In [24]:
# vamos utilizar um token da terceira sentença 
sent5 = sents[6]
sent5

Não vou gastar um minuto deste dia ensolarado para debater a interpretação rasteira, rasa mesmo, que certo jornalista fez do escravismo e da agência de mulheres escravizadas no Brasil.

In [25]:
len(sent5)

32

In [26]:
token1 = sent5[30]
token1

Brasil

In [27]:
# head = "pai" sintático do token1
token1.head

escravizadas

In [28]:
# entity type do token1
token1.ent_type

385

In [29]:
token1.ent_type_

'LOC'

In [30]:
# Lemma =  base lematizada do token1 (reduzir à base da palavra)
token2 = sent5[28]
print(token2.text, token2.lemma_)

escravizadas escravizar


In [31]:
# Análise morfológica
print(token2.text, token2.morph)

escravizadas Gender=Fem|Number=Plur|VerbForm=Part


In [32]:
# Classe gramatical = Part of Speech
print(token1.text, token1.pos_)
print(token2.text, token2.pos_)


Brasil PROPN
escravizadas VERB


Para ver todas as características linguísticas possíveis dos spaCy, veja o [link](https://spacy.io/usage/linguistic-features).

### Visualizar as relações entre tokens

In [35]:
from spacy import displacy

In [36]:
displacy.render(sent5, style="dep", minify=True, options={'distance': 110, 'compact': True})

## Entidades (Named Entities Recognition)

In [37]:
# imprimir as entidades do texto
for ent in doc.ents:
    print(ent.text, ent.label_)

Wlamyra Albuquerque PER
Folha LOC
Brasil LOC
Historiadoras PER
Leandro Narloch PER
Luxo PER
Folha LOC
Machado de Assis PER
Luiz Gama PER
Carolina de Jesus PER
Lima Barreto PER
Folha PER
rende likes PER
José do Patrocínio PER
Luiz Gama PER
Sim PER
terra LOC
Américas LOC
Brasil LOC
Amazônia LOC
Covid PER
Brasil LOC


In [38]:
displacy.render(doc, style="ent")

### Entidades Nomeadas do spaCy

Tipo de etiqueta|Descrição|
|:---:|:---:|
|PERSON|People, including fictional.|
|NORP|Nationalities or religious or political groups.|
|FAC|Buildings, airports, highways, bridges, etc.|
|ORG|Companies, agencies, institutions, etc.|
|GPE|Countries, cities, states.|
|LOC|Non-GPE locations, mountain ranges, bodies of water.|
|PRODUCT|Objects, vehicles, foods, etc. (Not services.)|
|EVENT|Named hurricanes, battles, wars, sports events, etc.|
|WORK_OF_ART|Titles of books, songs, etc.|
|LAW|Named documents made into laws.|
|LANGUAGE|Any named language.|
|DATE|Absolute or relative dates or periods.|
|TIME|Times smaller than a day.|
|PERCENT|Percentage, including ”%“.|
|MONEY|Monetary values, including unit.|
|QUANTITY|Measurements, as of weight or distance.|
|ORDINAL|“first”, “second”, etc.|
|CARDINAL|Numerals that do not fall under another type.|

In [39]:
# imprimiar apenas as entidades do tipo PERSON
for named_entity in doc.ents:
    if named_entity.label_ == "PER":
        print(named_entity)

Wlamyra Albuquerque
Historiadoras
Leandro Narloch
Luxo
Machado de Assis
Luiz Gama
Carolina de Jesus
Lima Barreto
Folha
rende likes
José do Patrocínio
Luiz Gama
Sim
Covid


In [40]:
# imprimir apenas as entidades do tipo VERB e seu lemma
for token in doc:
    if token.pos_ == 'VERB':
        print(token, token.lemma_)

anda andar
vende vender
anda andar
come comer
lê ler
disparam disparar
têm ter
gastar gastar
ensolarado ensolarado
debater debater
fez fazer
escravizadas escravizar
sentar-se sentar-se
marcados marcar
articuladas articular
temos ter
fundadas fundar
sinhás sinhá
precisam precisar
inspirar inspirar
passa passar
merecem merecer
Deixemos Deixemos
alcança alcançar
escravizadas escravizar
enxerga enxergar
embaçadas embaçar
peço pedir
informem-se informem-se
publicadas publicar
produzida produzir
abandonam abandonar
Basta Basta
corra correr
poderá poder
ler ler
dada dar
vale valer
perguntar perguntar
impresso impresso
abrir abrir
atiçar atiçar
vende vender
saber saber
abrir abrir
sustentam sustentar
enlutado enlutar
arriadas arriar
rende render
podia poder
sustentada sustentar
recusou recusar
sentar sentar
rendidos rendido
aderir aderir
questionar questionar
diziam dizer
defender defender
publicavam publicar
dizia dizer
sei saber
deve dever
haver haver
reivindicar reivindicar
desconfiar desco

## Vamos ver como funciona em inglês?

In [41]:
corpus = open('pos_1900.txt', 'r').read()

In [42]:
!python -m spacy download en_core_web_sm

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 13.6 MB 151 kB/s 
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [43]:
nlp_en = spacy.load('en_core_web_sm')

In [44]:
doc_en = nlp_en(corpus)

In [45]:
displacy.render(doc_en, style="ent")

### Encontrar as pessoas e contar

<small>Inspirado no exemplo de Melanie Walsh no curso [Introduction to cultural analytics & Python](https://melaniewalsh.github.io/Intro-Cultural-Analytics/05-Text-Analysis/Multilingual/Portuguese/02-Named-Entity-Recognition-Portuguese.html#get-people)</small>

In [46]:
import pandas as pd
from collections import Counter # para contar o número de ocorrências de cada palavra


In [47]:
# criar conjunto de textos divididos por quebra de linha
chunked_text = corpus.split('\n')
# passa a lista de textos para o pipeline de análise do Spacy
chunked_documents = list(nlp_en.pipe(chunked_text))

In [48]:
len(chunked_documents)

193

In [49]:
# selecionar todas os tokens com a classe POS 'PERSON' e armazenar em uma lista
people = []
for document in chunked_documents:
    for named_entity in document.ents:
        if named_entity.label_ == "PERSON":
            people.append(named_entity.text)

# contar o número de ocorrências de cada palavra
people_count = Counter(people)

# criar um dataframe com as palavras e seus respectivos números de ocorrências
df_people = pd.DataFrame(people_count.most_common(), columns=['character', 'count'])
df_people

,character,count
0,Lady Jerningham,2
1,Lady Fleming,2
2,Cronje,2
3,Owen,2
4,Ladies KID GLOVES,1
5,Sewing Twine,1
6,Jerningham,1
7,Robespierre,1
8,Danton,1
9,Bassen,1


In [50]:
# selecionar todas os tokens com a classe POS 'GPE' ou 'LOC' e armazenar em uma lista
places = []
for document in chunked_documents:
    for named_entity in document.ents:
        if named_entity.label_ == 'GPE' or  named_entity.label_ == 'LOC':
            places.append(named_entity.text)

places_count = Counter(places)

df_places = pd.DataFrame(places_count.most_common(), columns=['place', 'count'])
df_places

,place,count
0,France,4
1,San Humanité,3
2,Havana,2
3,Charlotte,2
4,Britain,2
5,Transvaal,2
6,Britannia,2
7,Trinidad,2
8,Australia,2
9,Barracks,1
